# WSJ portal web page scraper

## Mount drive

In [1]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Import libraries

In [2]:
# from bs4 import BeautifulSoup

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

import pandas as pd
import numpy  as np
import re
import datetime
import os,glob
import json
from json.decoder import JSONDecodeError
from tqdm import tqdm
import re


## Beautify

In [3]:
# makes the source tree format like

def beautify(url):
    hdr = {'User-Agent': 'Mozilla/30.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)

    return BeautifulSoup(page, "html.parser")

## Scrap

In [4]:
def get_links_and_headlines_lists(start_url):

  podcasty_jour = beautify(start_url)

  result = [] 
  for nr, link in enumerate(podcasty_jour.find_all('a')):
    if link.get('href').__contains__('https://www.wsj.com/podcasts/the-journal/'): # specific text which needs to be in the link
      if link.get('href') not in result:  # avoiding duplicates
        result.append(link.get('href'))
  headline=[]
  for nr, head in enumerate(podcasty_jour.find_all('p', attrs={'class': 'WSJTheme--description--2TYt9czY typography--text-sm--2TGEP_to '})): # in this class should be headline
    regex=re.compile("\">(.*?)<a class")
    match= re.findall(regex, str(head))
    if match not in headline: # avoiding duplicates
      headline.append(match)
    # print(nr+1, '--->', match)
  return result, headline

In [40]:
start_url= 'https://www.wsj.com/podcasts/the-journal?mod=podcasts_tile'
start_url_1= 'https://www.wsj.com/podcasts/wsj-the-future-of-everything?mod=podcasts_tile' 
start_url_2= 'https://www.wsj.com/podcasts/tech-news-briefing?mod=podcasts_tile'
# start_urls=['https://www.wsj.com/podcasts/the-journal?mod=podcasts_tile','https://www.wsj.com/podcasts/wsj-the-future-of-everything?mod=podcasts_tile']

In [ ]:
  # regex=re.compile("m/podcasts/(.*?)\?mod=") #this is getting from start_url the-journal
  # match= re.findall(regex, str(start_url_2))
  # dept = match[0]
  # dept

In [58]:
def get_links_and_headlines_lists(start_url):
  
  wsj= 'https://www.wsj.com/podcasts/'

  regex=re.compile("m/podcasts/(.*?)\?mod=") #this is getting from start_url the-journal
  match= re.findall(regex, str(start_url))
  dept = match[0]+"/"


  podcasty_jour = beautify(start_url)

  result = [] 
  for nr, link in enumerate(podcasty_jour.find_all('a')):
    if link.get('href').__contains__(wsj+dept): # specific text which needs to be in the link
      if link.get('href') not in result:  # avoiding duplicates
        result.append(link.get('href'))
  headline=[]
  for nr, head in enumerate(podcasty_jour.find_all('p', attrs={'class': 'WSJTheme--description--2TYt9czY typography--text-sm--2TGEP_to '})): # in this class should be headline
    regex=re.compile("\">(.*?)<a class")
    match= re.findall(regex, str(head))
    if match not in headline: # avoiding duplicates
      headline.append(match)
    # print(nr+1, '--->', match)

  nr_of_empty_string=len(result)-len(headline) # this number of default string will be appended to headline 
  if nr_of_empty_string > 0:
    for a in range (0,nr_of_empty_string):
      headline.append(['headline not scrapped sorry'])
  return result, headline

In [ ]:
result_jurnal, headline_jurnal=get_links_and_headlines_lists(start_url)
# result_jurnal, headline_jurnal

In [ ]:
result_future, headline_future=get_links_and_headlines_lists(start_url_1)
# result_future, headline_future

In [ ]:
result_tech, headline_tech=get_links_and_headlines_lists(start_url_2)
# result_tech, headline_tech

In [67]:
res_all=result_jurnal+ result_future+result_tech
head_all=headline_jurnal+headline_future+headline_tech

# Print results from lists

In [90]:
for nr, i in enumerate(res_all):
  print(str(nr).ljust(2),
        res_all[nr].ljust(150),
        head_all[nr])      

0  https://www.wsj.com/podcasts/the-journal/the-unraveling-of-the-murdaugh-dynasty/5C36B446-6648-49B1-8452-510A4EE04CB2                                   ["Over Labor Day weekend, an attempted murder was reported to police in Hampton County, S.C. involving the scion of a powerful local family. The victim, Alex Murdaugh, later said he attempted to stage his own murder to try to collect insurance money. WSJ's Valerie Bauerlein looks at this case and other events that threaten to undermine the Murdaugh dynasty."]
1  https://www.wsj.com/podcasts/the-journal/the-day-facebook-logged-off/86A5762A-C985-47F4-A7AA-14F05B6D0807                                              ["On Monday, Facebook, Instagram and WhatsApp went offline for billions of people around the world. To fix it, Facebook's employees had to physically drive to data centers to address the problem. WSJ's Robert McMillan explains the cascade of failures that caused it to happen."]
2  https://www.wsj.com/podcasts/the-journal/a-huawe

Update to tables library-pandas need that to avoid write errors

In [66]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3 MB 5.2 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


# Saving results to file

In [70]:
wsj_links_headlines = pd.DataFrame({'web_link' : res_all,
                               'headline' : head_all,
                               })

In [71]:
wsj_links_headlines.sample(3)

,web_link,headline
28,https://www.wsj.com/podcasts/tech-news-briefin...,[A key question in the criminal fraud trial of...
17,https://www.wsj.com/podcasts/wsj-the-future-of...,[headline not scrapped sorry]
8,https://www.wsj.com/podcasts/the-journal/china...,[Evergrande built homes for China's growing mi...


In [ ]:
# plots_olx_pd

In [75]:
# generating file name with timestamp info for later comparison
def gen_h5_file_with_timestamp(name='name'):
    timestamp = datetime.datetime.now()
    ts1 = timestamp.strftime("%A%d%B%Y%m")
    ts2 = timestamp.strftime("%H")
    ts3 = timestamp.strftime("%M")
    ts4 = timestamp.strftime("%S")
    file_name= name + ts1 +"-"+ ts2+"-"+ ts3+"-"+ ts4+".h5"
    return file_name


In [76]:
file_name= gen_h5_file_with_timestamp(name='wsj_links_headlines_')
file_name

'wsj_links_headlines_Monday11October202110-20-00-48.h5'

In [77]:
wsj_links_headlines.to_hdf('/content/drive/MyDrive/wsj/scrapped/'+file_name, key='df', mode='w') 

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['web_link', 'headline'], dtype='object')]

  encoding=encoding,


# Checking changes since last time

In [78]:
# preparing list of existing(previously created by scrapping) pandas linkedin .h5 files
search_dir = "/content/drive/MyDrive/wsj/scrapped/" #directory where from where h5 is read s
def list_per_time(search_dir=search_dir):
    files = [x for x in (filter(os.path.isfile, glob.glob(search_dir + "wsj*.h5"))) ] 
    files_sorted= sorted(files, key=os.path.getmtime)
    print(files_sorted[:10])
    return files_sorted

In [79]:
# loading last( creation time) two dfs for comparison
def read_two_last_dfs():
    files_sorted=list_per_time()
    f1=files_sorted[-2:-1]
    f2=files_sorted[-1:]
    df1=pd.read_hdf(f1[0], 'df')
    df2=pd.read_hdf(f2[0], 'df')
    return df1, df2

In [80]:
df1, df2=read_two_last_dfs()

['/content/drive/MyDrive/wsj/scrapped/wsj_links_headlines_Monday11October202110-19-58-19.h5', '/content/drive/MyDrive/wsj/scrapped/wsj_links_headlines_Monday11October202110-20-00-48.h5']


In [82]:
print('=============links to new headlines since last checking=============\n')
for x,item in enumerate(df2.web_link.values[~df2['web_link'].isin(df1['web_link'])]):
    print(x, '-->',item )

=============links to new headlines since last checking=============



Checking differece to 18/08/2021

In [85]:
new_plots=[]
df_ref = pd.read_hdf('/content/drive/MyDrive/wsj/scrapped/wsj_links_headlines_Monday11October202110-19-58-19.h5', 'df')
print('=============links to new headlines since 18/08/2021 =============\n')
for x,item in enumerate(df2.web_link.values[~df2['web_link'].isin(df_ref['web_link'])]):
  new_plots.append(item)
  print(x, '-->',item )

=============links to new headlines since 18/08/2021 =============



What will be diffrence if df1 and 2 were changed places (what dissapeared...)

In [86]:
dissapeared=[]
df_ref = pd.read_hdf('/content/drive/MyDrive/wsj/scrapped/wsj_links_headlines_Monday11October202110-19-58-19.h5', 'df')
print('=============those dissaperarewd since 18/06/2021 =============\n')
for x,item in enumerate(df_ref.web_link.values[~df_ref['web_link'].isin(df2['web_link'])]):
  dissapeared.append(item)
  print(x, '-->',item,  )
  print(df_ref[df_ref.web_link==item].Description.values)

=============those dissaperarewd since 18/06/2021 =============



Checking if new plots are only the change  from promoted?

In [87]:
for plot in dissapeared:
  for new_p in new_plots:
    if plot.__contains__(new_p):
      print('is-->',new_p ,'  --was--->',plot)

In [88]:
def cosmetic_ad_change_check():
  
  for plot in dissapeared:
    for new_p in new_plots:
      if plot.__contains__(new_p):
        print('is-->',new_p ,'  --was--->',plot)

# Report

In [89]:
#@title Overall report { display-mode: "form" }

df_ref_link = "/content/drive/MyDrive/wsj/scrapped/wsj_links_headlines_Monday11October202110-19-58-19.h5" #@param ["'/content/drive/MyDrive/olx-scrap/scraped/agro_plots_olx_Wednesday18August202108-10-13-54.h5'"] {allow-input: true}


df_ref = pd.read_hdf(df_ref_link, 'df')

new_plots=[]
dissapeared=[]
ti= df_ref_link[-26: -12]
print('=============links to new plots since {} ============='.format(ti))
for x,item in enumerate(df2.web_link.values[~df2['web_link'].isin(df_ref['web_link'])]):
  new_plots.append(item)
  print(x, '-->',item.ljust(90), df2[df2.web_link==item].time.values, df2[df2.web_link==item].Description.values )
  # print(df2[df2.web_link==item].Description.values)

print('\n','=============those dissapeared since {} ============='.format(ti))
for x,item in enumerate(df_ref.web_link.values[~df_ref['web_link'].isin(df2['web_link'])]):
  dissapeared.append(item)
  print(x, '-->',item.ljust(90), df_ref[df_ref.web_link==item].Description.values)
  # print(df_ref[df_ref.web_link==item].Description.values)

print('\n','Remarks:')
cosmetic_ad_change_check()


=============links to new plots since 1October202110 =============

 =============those dissapeared since 1October202110 =============

 Remarks:
